In [1]:
# reporting/quarterly_report/modules/granting.py
from __future__ import annotations

import logging, sqlite3, datetime
from pathlib import Path
from typing import List
import calendar
import pandas as pd
from datetime import date
from typing import List, Tuple,Union
import numpy as np
from great_tables import GT, loc, style, html
import re

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status, 
    load_report_params                   # to inspect results
)

from reporting.quarterly_report.utils import RenderContext, BaseModule
from reporting.quarterly_report.report_utils.granting_utils import enrich_grants, _ensure_timedelta_cols, _coerce_date_columns
from reporting.quarterly_report.utils import Database, RenderContext


import altair as alt
from altair_saver import save
import selenium.webdriver



# ─────────────────────────────────────────────────────────────
# 2) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")

init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn
report = 'Quarterly_Report'

CALLS_TYPES_LIST = ['STG', 'ADG', 'POC', 'COG', 'SYG', 'StG', 'CoG', 'AdG', 'SyG', 'PoC', 'CSA']
# ──────────────────────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────────────────────

def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year



def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)



def months_in_scope(cutoff: pd.Timestamp) -> list[str]:
    """
    Returns list of month names from January to last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).strftime("%B").tolist()

    return months

def determine_po_category(row):
    instrument = str(row.get('Instrument', '')).strip()
    topic = str(row.get('Topic', '')).strip()
    try:
        if topic and any(call_type in topic for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in topic).upper()
            return category
        elif instrument and any(call_type in instrument for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in instrument).upper()
            return category
        return ''
    except Exception as e:
        raise

def extract_project_number(row):
    """
    Extract project number from 'Inv Text' if 'v_check_payment_type' contains RP patterns,
    otherwise return original 'v_check_payment_type' value
    """
    payment_type = row['v_check_payment_type']
    inv_text = row['Inv Text']
    
    # Handle NaN values
    if pd.isna(payment_type):
        return payment_type
    
    # Convert to string to handle any data type
    payment_type_str = str(payment_type)
    
    # Check if the payment_type contains RP patterns:
    # - Original pattern: RP + number + = + FP/IP (e.g., RP4=FP, RP2=IP)
    # - New pattern: RP + number + - + FP/IP (e.g., RP4-FP, RP2-IP)
    rp_patterns = [
        r'RP\d+=(?:FP|IP)',  # Original pattern: RP4=FP, RP2=IP, etc.
        r'RP\d+-(?:FP|IP)'   # New pattern: RP4-FP, RP2-IP, etc.
    ]
    
    # Check if any of the RP patterns match
    has_rp_pattern = any(re.search(pattern, payment_type_str) for pattern in rp_patterns)
    
    if has_rp_pattern:
        # Extract the numerical part from Inv Text column
        if pd.notna(inv_text):
            inv_text_str = str(inv_text).strip()
            # Extract leading digits from Inv Text
            number_match = re.match(r'^(\d+)', inv_text_str)
            if number_match:
                return number_match.group(1)
        
        # If no number found in Inv Text, return original payment_type
        return payment_type
    
    # Return original v_check_payment_type if no RP pattern found
    return payment_type


def map_project_to_call_type(project_num, mapping_dict):
    # If it's a numeric string, try to convert and lookup
    try:
        # Try to convert to int for lookup
        numeric_key = int(project_num)
        if numeric_key in mapping_dict:
            return mapping_dict[numeric_key]
    except (ValueError, TypeError):
        # If conversion fails, it's a non-numeric string like 'EXPERTS'
        pass
    
    # Return original value if no match found
    return project_num

def map_call_type_with_experts(row, grant_map):
    """
    Map call_type based on project_number and Inv Parking Person Id
    """
    project_num = row['project_number']
    parking_person_id = row['Inv Parking Person Id']
    
    # First, try to map using grant_map (convert project_num to int if possible)
    try:
        numeric_key = int(project_num)
        if numeric_key in grant_map:
            return grant_map[numeric_key]
    except (ValueError, TypeError):
        pass
    
    # If project_number is 'EXPERTS', keep it as 'EXPERTS'
    if str(project_num).upper() == 'EXPERTS':
        return 'EXPERTS'
    
    # If not found in grant_map and not 'EXPERTS', check Inv Parking Person Id
    if pd.notna(parking_person_id):
        parking_person_str = str(parking_person_id).upper()
        if parking_person_str in ['KACZMUR', 'WALASOU']:
            return 'EXPERTS_C0'
    
    # Return original project_number if no conditions are met
    return project_num

# ──────────────────────────────────────────────────────────────
# constants – adapt whenever a file-alias changes
# ──────────────────────────────────────────────────────────────

INVOICES_ALIAS = "c0_invoices_summa"
CALLS_ALIAS = 'call_overview'


cutoff = pd.to_datetime("2025-04-15")
report_params = load_report_params(report_name=report, db_path=db_path)


table_colors = report_params.get('TABLE_COLORS', {})
BLUE = table_colors.get("BLUE", "#004A99")
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4")
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B")
SUB_TOTAL_BACKGROUND = table_colors.get("subtotal_background_color", "#E6E6FA")

df_inv = fetch_latest_table_data(conn, INVOICES_ALIAS, cutoff)
df_calls =  fetch_latest_table_data(conn, CALLS_ALIAS , cutoff)

df_calls['CALL_TYPE'] = df_calls.apply(determine_po_category, axis=1)

# Apply the function to create the new column
df_inv = df_inv.loc[df_inv['Inv Fin Document Type Desc'] == 'Expenditure Invoice']
# Keep the first occurrence of each unique combination
df_inv  = df_inv.drop_duplicates(subset=['Inv Supplier Invoice Key',   'Inv Reception Date (dd/mm/yyyy)'])
df_inv['project_number'] = df_inv.apply(extract_project_number, axis=1)

df_inv['Programme'] = np.where(df_inv['Official Budget Line'] == '01 02 01 01', 'HEU',
                      np.where(df_inv['Official Budget Line'] == '01 02 99 01', 'H2020', 
                          df_inv['Official Budget Line']))  # Keep original value if no match

# Assuming your DataFrame is called 'df'
grant_map = df_calls.set_index('Grant Number')['CALL_TYPE'].to_dict()

# Apply the mapping
df_inv['call_type'] = df_inv['project_number'].apply(lambda x: map_project_to_call_type(x, grant_map))
df_inv['call_type'] = df_inv.apply(lambda row: map_call_type_with_experts(row, grant_map), axis=1)

df_inv['Inv Reception Date (dd/mm/yyyy)'] = pd.to_datetime(
    df_inv['Inv Reception Date (dd/mm/yyyy)'], 
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

df_inv['Inv Creation Date (dd/mm/yyyy)'] = pd.to_datetime(
    df_inv['Inv Creation Date (dd/mm/yyyy)'], 
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

# Step 2: Calculate Time to Invoice (in days)
df_inv['Time_to_Invoice'] = (df_inv['Inv Creation Date (dd/mm/yyyy)'] - 
                            df_inv['Inv Reception Date (dd/mm/yyyy)']).dt.days

# Step 3: Create binary column for on-time registration (1 if <= 5 days, 0 otherwise)
df_inv['registered_on_time'] = (df_inv['Time_to_Invoice'] <= 5).astype(int)
valid_call_types = set(grant_map.values()) | {'EXPERTS', 'EXPERTS_C0'}

# Remove CSA if it exists in the valid_call_types
valid_call_types.discard('CSA')

df_filtered = df_inv[df_inv['call_type'].isin(valid_call_types)].copy()

quarter_dates = get_scope_start_end(cutoff=cutoff)
last_valid_date = quarter_dates[1]

df_filtered = df_filtered[
    df_filtered['Inv Reception Date (dd/mm/yyyy)'] <= last_valid_date
].copy()


def create_registration_pivot_table(df, programme_name):
    """
    Create a pivot table for a specific programme (H2020 or HEU)
    using registered_on_time column and counting Inv Supplier Invoice Key
    """
    # Filter for the specific programme
    prog_data = df[df['Programme'] == programme_name].copy()
    
    if len(prog_data) == 0:
        print(f"No data found for programme: {programme_name}")
        return pd.DataFrame()
    
    pivot_rows = []
    
    # GRANTS section - all call_types except EXPERTS and EXPERTS_C
    grants_data = prog_data[~prog_data['call_type'].isin(['EXPERTS', 'EXPERTS_C0'])].copy()
    
    if len(grants_data) > 0:
        # Individual grant types
        for call_type in sorted(grants_data['call_type'].unique()):
            ct_data = grants_data[grants_data['call_type'] == call_type]
            
            # Count total invoices using Inv Supplier Invoice Key
            total_invoices = ct_data['Inv Supplier Invoice Key'].count()
            
            # Count on-time invoices (registered_on_time = 1)
            on_time_count = ct_data[ct_data['registered_on_time'] == 1]['Inv Supplier Invoice Key'].count()
            
            # Count late invoices (registered_on_time = 0)
            late_count = ct_data[ct_data['registered_on_time'] == 0]['Inv Supplier Invoice Key'].count()
            
            # Calculate percentages
            on_time_pct = (on_time_count / total_invoices * 100) if total_invoices > 0 else 0
            late_pct = (late_count / total_invoices * 100) if total_invoices > 0 else 0
            
            pivot_rows.append({
                'Category': 'GRANTS',
                'Type': call_type,
                'No of Invoices': total_invoices,
                '% registered on time': f"{on_time_pct:.2f}%",
                '% registered late': f"{late_pct:.2f}%"
            })
        
        # GRANTS TOTAL
        total_grants = grants_data['Inv Supplier Invoice Key'].count()
        on_time_grants = grants_data[grants_data['registered_on_time'] == 1]['Inv Supplier Invoice Key'].count()
        late_grants = grants_data[grants_data['registered_on_time'] == 0]['Inv Supplier Invoice Key'].count()
        
        on_time_grants_pct = (on_time_grants / total_grants * 100) if total_grants > 0 else 0
        late_grants_pct = (late_grants / total_grants * 100) if total_grants > 0 else 0
        
        pivot_rows.append({
            'Category': 'GRANTS',
            'Type': 'TOTAL:',
            'No of Invoices': total_grants,
            '% registered on time': f"{on_time_grants_pct:.2f}%",
            '% registered late': f"{late_grants_pct:.2f}%"
        })
    
    # EXPERTS section - EXPERTS and EXPERTS_C
    experts_data = prog_data[prog_data['call_type'].isin(['EXPERTS', 'EXPERTS_C0'])].copy()
    
    if len(experts_data) > 0:
        # Individual expert types
        for expert_type in ['EXPERTS', 'EXPERTS_C0']:
            if expert_type in experts_data['call_type'].values:
                exp_data = experts_data[experts_data['call_type'] == expert_type]
                
                # Count total invoices using Inv Supplier Invoice Key
                total_experts = exp_data['Inv Supplier Invoice Key'].count()
                
                # Count on-time and late invoices
                on_time_experts = exp_data[exp_data['registered_on_time'] == 1]['Inv Supplier Invoice Key'].count()
                late_experts = exp_data[exp_data['registered_on_time'] == 0]['Inv Supplier Invoice Key'].count()
                
                # Calculate percentages
                on_time_exp_pct = (on_time_experts / total_experts * 100) if total_experts > 0 else 0
                late_exp_pct = (late_experts / total_experts * 100) if total_experts > 0 else 0
                
                pivot_rows.append({
                    'Category': 'EXPERTS',
                    'Type': expert_type,
                    'No of Invoices': total_experts,
                    '% registered on time': f"{on_time_exp_pct:.2f}%",
                    '% registered late': f"{late_exp_pct:.2f}%"
                })
        
        # EXPERTS TOTAL
        total_all_experts = experts_data['Inv Supplier Invoice Key'].count()
        on_time_all_experts = experts_data[experts_data['registered_on_time'] == 1]['Inv Supplier Invoice Key'].count()
        late_all_experts = experts_data[experts_data['registered_on_time'] == 0]['Inv Supplier Invoice Key'].count()
        
        on_time_all_exp_pct = (on_time_all_experts / total_all_experts * 100) if total_all_experts > 0 else 0
        late_all_exp_pct = (late_all_experts / total_all_experts * 100) if total_all_experts > 0 else 0
        
        pivot_rows.append({
            'Category': 'EXPERTS',
            'Type': 'TOTAL:',
            'No of Invoices': total_all_experts,
            '% registered on time': f"{on_time_all_exp_pct:.2f}%",
            '% registered late': f"{late_all_exp_pct:.2f}%"
        })
    
    # PROGRAMME TOTAL
    total_programme = prog_data['Inv Supplier Invoice Key'].count()
    on_time_programme = prog_data[prog_data['registered_on_time'] == 1]['Inv Supplier Invoice Key'].count()
    late_programme = prog_data[prog_data['registered_on_time'] == 0]['Inv Supplier Invoice Key'].count()
    
    on_time_prog_pct = (on_time_programme / total_programme * 100) if total_programme > 0 else 0
    late_prog_pct = (late_programme / total_programme * 100) if total_programme > 0 else 0
    
    pivot_rows.append({
        'Category': programme_name,
        'Type': 'TOTAL:',
        'No of Invoices': total_programme,
        '% registered on time': f"{on_time_prog_pct:.2f}%",
        '% registered late': f"{late_prog_pct:.2f}%"
    })
    
    return pd.DataFrame(pivot_rows)

# Create pivot tables for both programmes
df_h2020_pivot = create_registration_pivot_table(df_filtered, 'H2020')
df_heu_pivot = create_registration_pivot_table(df_filtered, 'HEU')

df_h2020 = df_h2020_pivot.reset_index(drop=True).copy()
df_heu  = df_heu_pivot.reset_index(drop=True).copy()

def create_registration_great_table_grouped(df_pivot, programme_name):
    """
    Create a great_table with row grouping by Category - BALANCED LAYOUT
    """
    
    # Prepare the dataframe for row grouping
    df_for_gt = df_pivot.copy()
    
    tbl = (
        GT(df_for_gt, rowname_col='Type', groupname_col='Category')
        .opt_table_font(font="Arial")
        .opt_table_outline(style="solid", width='1px', color="#cccccc") 
        .tab_options(
            table_font_size="12px",
            table_width="100%",
            table_background_color="#ffffff",
            table_font_color=DARK_BLUE
        )
        .tab_style(
            style=style.borders(sides="all", color="#cccccc", weight="1px"),
            locations=loc.body()
        )
        
        # Row group styling - BALANCED
        .tab_style(
            style=[
                style.text(color=DARK_BLUE, weight="bold", size='medium'),
                style.fill(color=LIGHT_BLUE),
                style.css("line-height:1.2; padding:5px; width:25%;")  # Equal 25% width
            ],
            locations=loc.row_groups()
        )
        
        # Column labels styling - BALANCED
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align="center", size='small'),
                style.css("padding:8px; line-height:1.2; width:25%;")  # Equal 25% width for each column
            ],
            locations=loc.column_labels()
        )
        
        # Stubhead styling - DARK BLUE & BALANCED
        .tab_style(
            style=[
                style.fill(color=BLUE),
                style.text(color="white", weight="bold", align="center", size='small'),
                style.css("padding:8px; line-height:1.2; width:25%;")  # Equal 25% width
            ],
            locations=loc.stubhead()
        )
        
        # Body cell styling - BALANCED
        .tab_style(
            style=[
                style.text(align="center", size='small'),
                style.css("padding:8px; width:25%;")  # Equal 25% width
            ],
            locations=loc.body()
        )
        
        # Stub cell styling - BALANCED
        .tab_style(
            style=[
                style.text(size='small', align="left"),
                style.css("padding:8px; width:25%;")  # Equal 25% width
            ],
            locations=loc.stub()
        )
        
        # TOTAL rows styling
        .tab_style(
            style=[
                style.fill(color=SUB_TOTAL_BACKGROUND),
                style.text(color=DARK_BLUE, weight="bold")
            ],
            locations=loc.body(
                rows=lambda df: df['Type'].apply(lambda x: 'TOTAL:' in str(x))
                
            )
        )

         .tab_style(
            style=[
                style.fill(color=SUB_TOTAL_BACKGROUND),
                style.text(color=DARK_BLUE, weight="bold")
            ],
            locations=loc.stub(
                rows=lambda df: df['Type'].apply(lambda x: 'TOTAL:' in str(x))
                
            )
        )
        
        # Column labels with balanced spacing
        .cols_label(**{
            'No of Invoices': html(
                "<span style='display:block; text-align:center;'>No of</span>"
                "<span style='display:block; text-align:center;'>Invoices</span>"
            ),
            '% registered on time': html(
                "<span style='display:block; text-align:center;'>%</span>"
                "<span style='display:block; text-align:center;'>registered</span>"
                "<span style='display:block; text-align:center;'>on time</span>"
            ),
            '% registered late': html(
                "<span style='display:block; text-align:center;'>%</span>"
                "<span style='display:block; text-align:center;'>registered</span>"
                "<span style='display:block; text-align:center;'>late</span>"
            )
        })
        
        # Add title with BOLD styling
        .tab_header(
            title=html(f"<b>{programme_name} - Proportion of Invoices registered within 5 working days</b>")
        )
        
        # Title styling - BOLD
        .tab_style(
            style=[
                style.text(weight="bold", size="large", align="center"),
                style.css("padding:10px;")
            ],
            locations=loc.title()
        )
        
         # Footer - CONSISTENT WHITE BACKGROUND
        .tab_source_note("Source: SUMMA DWH")
        .tab_source_note("Report: C0_INVOICES_SUMMA")
        .tab_style(
            style=[
                style.text(size="small", color=DARK_BLUE),
                style.fill(color="#ffffff"),  # White background for consistent color
                style.css("padding:5px; line-height:1.2; border:none;")  # Remove borders if needed
            ],
            locations=loc.footer()
        )
    )
    
    return tbl

# Create grouped tables with CORRECT DataFrame names
tbl_h2020_grouped = create_registration_great_table_grouped(df_h2020_pivot, "H2020")
tbl_heu_grouped = create_registration_great_table_grouped(df_heu_pivot, "HEU")


DEBUG:root:Fetching latest data for table_alias: c0_invoices_summa, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for c0_invoices_summa: [('2025-06-02T06:44:39.967534', 1)]
DEBUG:root:Checking upload_id: 1, uploaded_at: 2025-06-02T06:44:39.967534
DEBUG:root:Fetched 5658 rows from c0_invoices_summa with upload_id 1
DEBUG:root:Fetching latest data for table_alias: call_overview, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for call_overview: [('2025-06-02T07:10:49.675162', 2)]
DEBUG:root:Checking upload_id: 2, uploaded_at: 2025-06-02T07:10:49.675162
DEBUG:root:Fetched 13295 rows from call_overview with upload_id 2


In [2]:
tbl_heu_grouped.show()

HEU - Proportion of Invoices registered within 5 working days 
 
 
 
 No of Invoices 
 % registered on time 
 % registered late 
 
 
 
 
 GRANTS 
 
 
 ADG 
 12 
 16.67% 
 83.33% 
 
 
 COG 
 22 
 22.73% 
 77.27% 
 
 
 POC 
 67 
 1.49% 
 98.51% 
 
 
 STG 
 93 
 6.45% 
 93.55% 
 
 
 SYG 
 15 
 0.00% 
 100.00% 
 
 
 TOTAL: 
 209 
 6.70% 
 93.30% 
 
 
 EXPERTS 
 
 
 EXPERTS 
 1662 
 49.70% 
 50.30% 
 
 
 EXPERTS_C0 
 15 
 6.67% 
 93.33% 
 
 
 TOTAL: 
 1677 
 49.31% 
 50.69% 
 
 
 HEU 
 
 
 TOTAL: 
 1886 
 44.59% 
 55.41% 
 
 
 
 
 
 Source: SUMMA DWH 
 


 
 Report: C0_INVOICES_SUMMA